# Example usage with the CLI
This notebook demonstrates usage of `petab_select` to perform model selection with commands.

Note that the criterion values in this notebook are for demonstrative purposes only, and are not real (the models were not calibrated).

In [1]:
# Cleanup the state and candidate models output by a previous run of this notebook
import os

try:
    os.remove('output/state.dill')
except FileNotFoundError:
    pass

try:
    os.remove('output/models.yaml')
except FileNotFoundError:
    pass

## First iteration

In each call to `petab_select search`, the following options are required:
- `-y`: The PEtab Select problem YAML file;
- `-s`: A file that is used to stored the state of the problem (e.g., such that models are not repeated);
- `-o`: A file to store the output of the model space search; and
- `-m`: The method used to search or identify models in the model space.

Other options can be viewed with `petab_select search --help`.

In this initial call, a PEtab Select problem is used to identify possible models for selection. The brute force method is used, which normally outputs all possible models. Here, the number of models in the output is explicitly limited to `3`. Subsequent calls with the same command will output different models.

In [2]:
%%bash

petab_select search \
-y model_selection/selection_problem.yaml \
-s output/state.dill \
-o output/models.yaml \
-m brute_force \
-l 3

The output format is a list of the PEtab Select model YAML format.

In [3]:
with open('output/models.yaml', 'r') as f:
    print(f.read())

- criteria: null
  model_id: M1_0_0
  parameters:
    k1: 0.0
    k2: 0.0
    k3: 0.0
  petab_yaml: model_selection/example_modelSelection.yaml
- criteria: null
  model_id: M1_1_0
  parameters:
    k1: 0.2
    k2: 0.1
    k3: estimate
  petab_yaml: model_selection/example_modelSelection.yaml
- criteria: null
  model_id: M1_2_0
  parameters:
    k1: 0.2
    k2: estimate
    k3: 0.0
  petab_yaml: model_selection/example_modelSelection.yaml



## Second iteration

The models from the first iteration have been calibrated, and the model with the best criterion value is `M1_1_0`.

In [4]:
with open('model_selection/calibrated/M1_1_0.yaml', 'r') as f:
    print(f.read())

criteria:
  AIC: 100
model_id: M1_1_0
parameters:
  k1: 0.2
  k2: 0.1
  k3: estimate
estimated_parameters:
  k3: 0.0
petab_yaml: ../model_selection/example_modelSelection.yaml



This is used as the initial model for the next search, this time using the forward method. The same state file can be used in each call. If the output file already exists, it will be overwritten.

In [5]:
%%bash

petab_select search \
-y model_selection/selection_problem.yaml \
-s output/state.dill \
-o output/models.yaml \
-m forward \
-i model_selection/calibrated/M1_1_0.yaml

/home/dilan/Documents/future_annex/model_selection/packages/draft_model_selection_extension/petab_select/model.py:165: UserWarning: Ignoring unknown attributes: estimated_parameters
  warnings.warn(


`M1_1_0` has one estimated parameter, `k3` [1]. As expected, the new candidates identified with the forward method have two estimated parameters, and one of them is `k3`.

[1] There may be additional estimated parameters specified in the PEtab problem.

In [6]:
with open('output/models.yaml', 'r') as f:
    print(f.read())

- criteria: null
  model_id: M1_4_0
  parameters:
    k1: 0.2
    k2: estimate
    k3: estimate
  petab_yaml: model_selection/example_modelSelection.yaml
- criteria: null
  model_id: M1_5_0
  parameters:
    k1: estimate
    k2: 0.1
    k3: estimate
  petab_yaml: model_selection/example_modelSelection.yaml



## Third iteration

Models `M1_4_0` and `M1_5_0` were calibrated, with `M1_4_0` having the superior criterion value, so is used as the initial model in the next forward search.

In [7]:
with open('model_selection/calibrated/M1_4_0.yaml', 'r') as f:
    print(f.read())

criteria:
  AIC: 50
model_id: M1_4_0
parameters:
  k1: 0.2
  k2: estimate
  k3: estimate
estimated_parameters:
  k2: 0.15
  k3: 0.0
petab_yaml: ../model_selection/example_modelSelection.yaml



In [8]:
%%bash

petab_select search \
-y model_selection/selection_problem.yaml \
-s output/state.dill \
-o output/models.yaml \
-m forward \
-i model_selection/calibrated/M1_4_0.yaml

/home/dilan/Documents/future_annex/model_selection/packages/draft_model_selection_extension/petab_select/model.py:165: UserWarning: Ignoring unknown attributes: estimated_parameters
  warnings.warn(


The model space contains only one model with 3 or more estimated parameters, which is a valid candidate model in this case.

In [9]:
with open('output/models.yaml', 'r') as f:
    print(f.read())

- criteria: null
  model_id: M1_7_0
  parameters:
    k1: estimate
    k2: estimate
    k3: estimate
  petab_yaml: model_selection/example_modelSelection.yaml



## Fourth iteration
As there are no models in the model space with additional estimated parameters, subsequent forward searches will return no candidate models.

In [10]:
with open('model_selection/calibrated/M1_7_0.yaml', 'r') as f:
    print(f.read())

criteria:
  AIC: 40
model_id: M1_7_0
parameters:
  k1: estimate
  k2: estimate
  k3: estimate
estimated_parameters:
  k1: 0.25
  k2: 0.1
  k3: 0.0
petab_yaml: ../model_selection/example_modelSelection.yaml



In [11]:
%%bash

petab_select search \
-y model_selection/selection_problem.yaml \
-s output/state.dill \
-o output/models.yaml \
-m forward \
-i model_selection/calibrated/M1_7_0.yaml

/home/dilan/Documents/future_annex/model_selection/packages/draft_model_selection_extension/petab_select/model.py:165: UserWarning: Ignoring unknown attributes: estimated_parameters
  warnings.warn(


In [12]:
with open('output/models.yaml', 'r') as f:
    print(f.read())

null
...



## Fifth iteration
Although no additional models are found with a forward search initialized at the best model so far (`M1_7_0`), there are additional models in the model space that are yet to be calibrated, which can be identified with the brute force method.

In [13]:
%%bash

petab_select search \
-y model_selection/selection_problem.yaml \
-s output/state.dill \
-o output/models.yaml \
-m brute_force

In [14]:
with open('output/models.yaml', 'r') as f:
    print(f.read())

- criteria: null
  model_id: M1_3_0
  parameters:
    k1: estimate
    k2: 0.1
    k3: 0.0
  petab_yaml: model_selection/example_modelSelection.yaml
- criteria: null
  model_id: M1_6_0
  parameters:
    k1: estimate
    k2: estimate
    k3: 0.0
  petab_yaml: model_selection/example_modelSelection.yaml

